In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
# Create empty lists to hold table data to be scraped
away_box_score = []
home_box_score = []
box_score = []

# input URL and use BeautifulSoup to parse through the page
url = f'https://www.basketball-reference.com/boxscores/202212190SAC.html'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

# Grab the table element that has away team statistics
away_table = soup.select_one('#div_box-CHO-game-basic') 

# Grab data from table and put it into the list created above
for tr in away_table.select('tr:has(td)'):
    tds = [td.get_text(strip=True) for td in tr.select('td')]
    away_box_score.append(tds)

# Grab the table element that has home team statistics
home_table = soup.select_one('#div_box-SAC-game-basic') 

# Grab data from table and put it into the list created above
for tr in home_table.select('tr:has(td)'):
    tds2 = [td.get_text(strip=True) for td in tr.select('td')]
    home_box_score.append(tds2)

# Grab the table element that has game box score
box_score_table = soup.select_one('#all_line_score')

# Game box score is commented out in html, so this will grab it out of the comments
for comment in box_score_table.find_all(text=lambda text: isinstance(text, Comment)):
    if comment.find("<table ") > 0:
        comment_soup = BeautifulSoup(comment, 'html.parser')
        table = comment_soup.find("table")

# Grab data from table and put it into the list created above
for tr in table.select('tr:has(td)'):
    tds3 = [td.get_text(strip=True) for td in tr.select('td')]
    box_score.append(tds3)


In [3]:
# Create dataframe for away team statistics
away_team_df = pd.DataFrame(away_box_score)

# Create an empty list to store away team statistics header information
away_header_list = []

# Grab the table header information to use as column headers in our away team statistics dataframe
for tr in away_table.select('tr:has(th)'):
    ths = [th.get_text(strip=True) for th in tr.select('th')]
    away_header_list.append(ths)
    
# Create dataframe for home team statistics
home_team_df = pd.DataFrame(home_box_score)

# Create an empty list to store home team statistics header information
home_header_list = []

# Grab the table header information to use as column headers in our home team statistics dataframe
for tr in home_table.select('tr:has(th)'):
    ths2 = [th.get_text(strip=True) for th in tr.select('th')]
    home_header_list.append(ths2)
    
# Create dataframe for game box score
box_score_df = pd.DataFrame(box_score)

# Create an empty list to store game box score dataframe header information
box_score_list = []

# Grab the table header information to use as column headers in our game box score dataframe
for tr in box_score_table.select('tr:has(th)'):
    ths3 = [th.get_text(strip=True) for th in tr.select('th')]
    box_score_list.append(ths3)

In [4]:
# Update the column and row labels
box_score_df = box_score_df.set_axis(['1Q', '2Q', '3Q', '4Q', 'T'], axis=1, inplace=False)
box_score_df = box_score_df.set_axis(['Away', 'Home'])

In [5]:
# Pull the away team player data and combine the lists into one list
away_starters = away_header_list[2:7]
away_bench = away_header_list[8:-1]
away_players = [away_starters,away_bench]

# Pull the home team player data and combine the lists into one list
home_starters = home_header_list[2:7]
home_bench = home_header_list[8:-1]
home_players = [home_starters,home_bench]

In [6]:
# Create a dataframe from away team stats
away_stats_df = pd.DataFrame(away_box_score)

# Create a dataframe from home team stats
home_stats_df = pd.DataFrame(home_box_score)

In [7]:
# Consolodate the away team list of lists into one list
away_starters_list = [item for sublist in away_starters for item in sublist]
away_bench_list = [item for sublist in away_bench for item in sublist]
away_roster = away_starters_list + away_bench_list

# Consolodate the home team list of lists into one list
home_starters_list = [item for sublist in home_starters for item in sublist]
home_bench_list = [item for sublist in home_bench for item in sublist]
home_roster = home_starters_list + home_bench_list

In [8]:
# Add Team Totals to the end of the away team list
away_roster.append('Team Totals')
away_stats_df.index = away_roster

# Add Team Totals to the end of the home team list
home_roster.append('Team Totals')
home_stats_df.index = home_roster

In [9]:
# Set the home and away dataframe column labels
column_headers = away_header_list[1]
column_headers.remove("Starters")
away_stats_df.columns = column_headers
home_stats_df.columns = column_headers

In [10]:
away_stats_df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Kelly Oubre Jr.,38:27,12,25,.480,3,8,.375,4,6,.667,1,8,9,1,1,1,4,2,31,+1
Gordon Hayward,36:43,8,12,.667,1,2,.500,2,2,1.000,1,4,5,1,1,0,4,2,19,+18
P.J. Washington,29:03,1,6,.167,1,4,.250,0,0,,1,8,9,2,0,1,0,5,3,-5
LaMelo Ball,27:28,9,19,.474,5,10,.500,0,0,,1,4,5,12,0,0,2,6,23,+3
Mason Plumlee,25:08,7,8,.875,0,0,,1,2,.500,0,4,4,3,0,3,1,3,15,-4
Théo Maledon,26:04,4,10,.400,0,3,.000,4,6,.667,0,7,7,3,1,0,0,2,12,+6
Jalen McDaniels,25:50,1,5,.200,0,4,.000,4,4,1.000,1,2,3,2,0,2,2,2,6,+3
Nick Richards,22:52,6,9,.667,0,0,,2,3,.667,5,6,11,2,1,0,1,3,14,+10
JT Thor,8:24,1,1,1.000,0,0,,0,0,,1,2,3,0,0,0,0,0,2,-2
James Bouknight,Did Not Play,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [11]:
home_stats_df

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Domantas Sabonis,40:44,12,19,.632,2,2,1.000,2,5,.400,5,18,23,7,1,0,1,4,28,+6
De'Aaron Fox,33:43,14,25,.560,2,5,.400,7,9,.778,1,4,5,2,0,0,2,5,37,-20
Kevin Huerter,31:47,3,11,.273,3,8,.375,0,2,.000,1,2,3,1,0,0,0,5,9,+6
Keegan Murray,31:02,4,9,.444,3,5,.600,0,2,.000,0,1,1,0,1,1,0,4,11,+8
Harrison Barnes,29:29,3,7,.429,0,2,.000,5,8,.625,0,3,3,3,2,0,1,1,11,+2
Malik Monk,24:51,4,13,.308,1,6,.167,3,4,.750,0,1,1,4,0,0,2,1,12,-4
Davion Mitchell,21:04,3,4,.750,1,2,.500,0,0,,1,0,1,5,1,0,0,0,7,+1
Terence Davis,14:43,1,4,.250,0,3,.000,0,1,.000,0,0,0,1,3,0,0,1,2,-11
Neemias Queta,7:16,1,3,.333,0,0,,0,0,,2,2,4,0,0,1,1,0,2,-12
Trey Lyles,5:20,0,0,,0,0,,0,0,,0,1,1,0,0,0,1,2,0,-6


In [12]:
box_score_df

,1Q,2Q,3Q,4Q,T
Away,39,27,25,34,125
Home,30,33,29,27,119
